# **RECAP - Fundamentos de Ciência de Dados**

**Diretórios Brasileiros**

https://basedosdados.org/dataset/br-bd-diretorios-brasil?bdm_table=municipio

SELECT * FROM `basedosdados.br_bd_diretorios_brasil.municipio`

<br>

**Índice Nacional de Preços ao Consumidor Amplo (IPCA)**

https://basedosdados.org/dataset/br-ibge-ipca?bdm_table=mes_categoria_municipio

SELECT * FROM `basedosdados.br_ibge_ipca.mes_categoria_municipio`

<br>

**População Brasileira**

https://basedosdados.org/dataset/br-ibge-populacao?bdm_table=municipio

SELECT * FROM `basedosdados.br_ibge_populacao.municipio`

### **Fazendo a conexão com o BigQuery para juntar as tabelas**

In [82]:
#Importando as bibliotecas

import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account

In [83]:
#credencial que permite se conectar ao bigquery
credencial = service_account.Credentials.from_service_account_file('/content/atividade-1-353400-a2b0f4c06d01.json')

#permite executar querys no bigquery
client = bigquery.Client(credentials= credencial, project = credencial.project_id)

## **PARTE 1**

Criando uma query que faça a junção das tabelas `basedosdados.br_bd_diretorios_brasil.municipio` e `basedosdados.br_ibge_ipca.mes_categoria_municipio`.

Obs: Filtrando apenas as colunas id_municipio, nome e sigla_uf da tabela `basedosdados.br_bd_diretorios_brasil.municipio` e ano, mes, variacao_mensal e variacao_anual da tabela `basedosdados.br_ibge_ipca.mes_categoria_municipio`.

In [84]:
df = pd.read_gbq('''
  SELECT
    municipio.id_municipio,
    nome,
    municipio.sigla_uf,
    ano,
    mes,
    variacao_mensal,
    variacao_anual
  FROM
    basedosdados.br_bd_diretorios_brasil.municipio AS municipio
  INNER JOIN
    basedosdados.br_ibge_ipca.mes_categoria_municipio AS ibge
  ON
    municipio.id_municipio = ibge.id_municipio
    ''', credentials = credencial)

Visualizando os dados

In [85]:
df.head()

,id_municipio,nome,sigla_uf,ano,mes,variacao_mensal,variacao_anual
0,1200401,Rio Branco,AC,2021,9,1.56,8.16
1,1200401,Rio Branco,AC,2022,2,0.93,1.80
2,1200401,Rio Branco,AC,2020,2,0.49,0.28
3,1200401,Rio Branco,AC,2020,7,0.75,0.40
4,1200401,Rio Branco,AC,2022,4,0.90,4.11


Verificando a quantidade de registros no conjunto de dados

In [86]:
df.shape

(90486, 7)

Verificando se a tabela possui valores ausentes, caso exista, escreva uma função que identifica apenas as colunas que possuem 30% ou mais de valores ausentes. A função também deve permitir a exclusão das linhas que possuem valores ausentes nas colunas identificadas.**


A função deve receber como parâmetro o conjunto de dados e retornar um novo dataset.

Escreva um código que automatize a tarefa.

In [87]:
df.isnull().sum()

id_municipio           0
nome                   0
sigla_uf               0
ano                    0
mes                    0
variacao_mensal    30195
variacao_anual     30195
dtype: int64

In [88]:
def dropNulos(df):
  colunas = df.columns
  for i in df.columns:
    if df[i].isnull().sum()/len(df) > 0.3:
      df.dropna(subset=[str(i)], inplace = True)
  return df

In [89]:
df = dropNulos(df)

In [90]:
df.isnull().sum()

id_municipio       0
nome               0
sigla_uf           0
ano                0
mes                0
variacao_mensal    0
variacao_anual     0
dtype: int64

**Criando uma função que imprime na tela a quantidade de valores únicos em cada coluna.**

A função recebe como parâmetro o conjunto de dados.


In [91]:
def valoresUnicos(df):
  return df.nunique()

In [92]:
valoresUnicos(df)

id_municipio          6
nome                  6
sigla_uf              6
ano                   3
mes                  12
variacao_mensal    2883
variacao_anual     5439
dtype: int64

**4 - Descubra em qual mes há uma maior variação mensal do IPCA em cada municipio e em cada ano?**

O que você conseguiu observar?

In [93]:
df.groupby(['id_municipio', 'ano'])[['variacao_mensal', 'mes']].agg([np.max])

variacao_mensal  mes
                             amax amax
id_municipio ano                      
1200401      2020           47.29   12
             2021           34.26   12
             2022           29.04    9
2111300      2020           46.83   12
             2021           35.95   12
             2022           51.34    9
2800308      2020           56.79   12
             2021           57.46   12
             2022           47.56    9
5002704      2020           58.95   12
             2021           51.75   12
             2022           88.62    9
5208707      2020           46.00   12
             2021           42.18   12
             2022           62.04    9
5300108      2020           56.32   12
             2021           41.90   12
             2022           62.12    9

**Salvando o conjunto de dados tratado no bigquery**

In [94]:
df.to_gbq(credentials=credencial, destination_table="atividade_1_353400.df", if_exists="replace")

1it [00:04,  4.69s/it]


**Escrevendo uma query para filtrar a média da variação anual do IPCA em cada munícipio. A query deve ordenar os valores de acordo com a média da variação anual.**

In [95]:
df2 = pd.read_gbq(''' 
  SELECT 
  municipio.id_municipio,
  ROUND(AVG(variacao_anual),2) as media_variacao_anual,
  FROM basedosdados.br_bd_diretorios_brasil.municipio as municipio
  INNER JOIN basedosdados.br_ibge_ipca.mes_categoria_municipio as ibge
  ON municipio.id_municipio = ibge.id_municipio
  GROUP BY municipio.id_municipio
  ORDER BY media_variacao_anual DESC
''', credentials = credencial)

In [96]:
df2

,id_municipio,media_variacao_anual
0,2800308,4.17
1,1200401,4.07
2,2111300,3.85
3,5300108,3.80
4,5002704,3.78
5,5208707,3.29


## **PARTE 2**

**ATENÇÃO: Carregue os arquivos transacoes.csv e clientes.csv e depois salve no BigQuery**

In [100]:
clientes = pd.read_csv('https://raw.githubusercontent.com/daniel-arnobio/datasets/main/clientes%20(1).csv')

In [101]:
clientes.to_gbq(credentials=credencial, destination_table="atividade_1_353400.clientes", if_exists="replace")

1it [00:02,  2.95s/it]


In [98]:
transacoes = pd.read_csv('https://raw.githubusercontent.com/daniel-arnobio/datasets/main/transacoes%20(2).csv')

In [102]:
transacoes.to_gbq(credentials=credencial, destination_table="atividade_1_353400.transacoes", if_exists="replace")

1it [00:04,  4.36s/it]


1 - Na tabela transações, temos os envolvidos nas transações econômicas dentro de um aplicativo financeiro. 

* Criando um resumo dessa tabela contendo o valor total que cada usuário enviou pela plataforma.
* A tabela final deve conterá uma coluna chamada id e uma chamada total_transacionado, com o total de dinheiro que o usuário enviou pelo aplicativo - quem envia dinheiro é o usuario_origem.


In [103]:
df3 = pd.read_gbq('''
  SELECT
    usuario_origem AS id, 
    SUM(montante) AS total_transacionado,
  FROM
    `atividade-1-353400.atividade_1_353400.transacoes`
  GROUP BY
    id
  ORDER BY 
    total_transacionado DESC
''', credentials = credencial)

In [104]:
df3

,id,total_transacionado
0,50,5105.25
1,12,1600.20
2,16,824.00
3,17,740.50
4,40,699.00
5,36,584.70
6,5,555.25
7,31,550.00
8,51,500.10
9,42,444.10


**Reportando os clientes que mais enviaram dinheiro pela plataforma, através de uma consulta que retorna somente o nome dos 3 clientes que transacionaram as maiores quantias em dinheiro pela plataforma.**



In [105]:
df4 = pd.read_gbq('''
SELECT
    cliente.nome,
    transacoes.usuario_origem AS id, 
    SUM(montante) AS total_transacionado,
  FROM
    atividade-1-353400.atividade_1_353400.transacoes as transacoes
  INNER JOIN atividade-1-353400.atividade_1_353400.clientes as cliente
  ON transacoes.usuario_origem = cliente.id_usuario
  GROUP BY
    id,
    cliente.nome
  ORDER BY 
    total_transacionado DESC
''', credentials = credencial
)

In [107]:
df4.head(5)

,nome,id,total_transacionado
0,Weber Jardim,50,5105.25
1,Bruno Ferreira,12,1600.20
2,Denise Justino,16,824.00
3,Diego Lima,17,740.50
4,Raquel Rodrigues,40,699.00


**Criando uma consulta contendo os nomes dos 3 clientes que realizaram o maior número de transações. Atenção: neste caso, não estamos falando dos maiores valores transacionados, mas sim do maior número de transações.**


In [108]:
df5 = pd.read_gbq('''
  SELECT 
  cliente.nome,
  usuario_origem,
  COUNT(*) as qtde_transacoes,
  FROM
    atividade-1-353400.atividade_1_353400.transacoes as transacoes
    INNER JOIN atividade-1-353400.atividade_1_353400.clientes as cliente
    ON transacoes.usuario_origem = cliente.id_usuario
  GROUP BY usuario_origem,
  cliente.nome
  ORDER BY qtde_transacoes DESC
  LIMIT 3
''', credentials = credencial
)

In [109]:
df5

,nome,usuario_origem,qtde_transacoes
0,Alessandro Freire,3,4
1,Alana Leite,1,3
2,Matheus Barana,36,3


## **PARTE 3 - Extra**

Criando uma View Analítica no BigQuery

A view filtrar as colunas sigla_uf, id_municipio, admitidos_desligados, idade, sexo e raça_cor da tabela `basedosdados.br_me_caged.microdados_antigos` no ano de 2019 com limite de 30000 registros.

In [117]:
'''CREATE VIEW atividade-1-353400.atividade.df as (
  SELECT
    sigla_uf,
    id_municipio,
    admitidos_desligados,
    idade,
    sexo,
    raca_cor
  FROM `basedosdados.br_me_caged.microdados_antigos`
  WHERE ano=2019
  LIMIT 30000)
'''

'CREATE VIEW atividade-1-353400.atividade.df as (\n  SELECT\n    sigla_uf,\n    id_municipio,\n    admitidos_desligados,\n    idade,\n    sexo,\n    raca_cor\n  FROM `basedosdados.br_me_caged.microdados_antigos`\n  WHERE ano=2019\n  LIMIT 30000)\n'

 **Query que filtra a quantidade de pessoas admitidas e desligadas do emprego em cada estado.**

In [112]:
df7 = pd.read_gbq('''
SELECT
  COUNT(admitidos_desligados) AS qtd_admitidos_desligados,
  sigla_uf
FROM
  `atividade-1-353400.atividade.df`
GROUP BY
  sigla_uf''', credentials = credencial
)

In [113]:
df7

,qtd_admitidos_desligados,sigla_uf
0,7430,AP
1,15091,PI
2,7479,TO


**Query que filtra a quantidade de pessoas admitidas e desligadas do emprego por sexo em cada estado.**

Alterando os valores da coluna Sexo para texto. 01 é Feminino e 02 é Masculino.

In [114]:
df8 = pd.read_gbq(''' SELECT
  sigla_uf,
  COUNT(admitidos_desligados) AS qtd_admitidos_desligados,
CASE 
  WHEN sexo = '01' THEN 'f'
  WHEN sexo = '02' THEN 'm'
END as genero
FROM
  `atividade-1-353400.atividade.df`
GROUP BY
  sexo,
  sigla_uf''', credentials = credencial)

In [115]:
df8

,sigla_uf,qtd_admitidos_desligados,genero
0,AP,4933,f
1,AP,2497,m
2,AC,5098,f
3,AC,3120,m
4,TO,3808,m
5,TO,8143,f
6,PI,1816,f
7,PI,585,m
